In [3]:
import os
import pandas as pd

In [4]:
propilot_path = os.path.join("data", "data_propilot")

In [30]:
df_effects = pd.read_csv(os.path.join(propilot_path, "dim_effects_202101281645.csv"), sep=";")
df_maturities = pd.read_csv(os.path.join(propilot_path, "dim_maturities_202101281645.csv"), sep=";")
df_periods = pd.read_csv(os.path.join(propilot_path, "dim_periods_202101281645.csv"), sep=";")
df_properties = pd.read_csv(os.path.join(propilot_path, "dim_properties_202101281645.csv"), sep=";")
df_snapshots = pd.read_csv(os.path.join(propilot_path, "dim_snapshots_202101281645.csv"), sep=";")
df_states = pd.read_csv(os.path.join(propilot_path, "dim_states_202101281645.csv"), sep=";")
df_structures = pd.read_csv(os.path.join(propilot_path, "dim_structures_202101281645.csv"), sep=";")
df_top_levels = pd.read_csv(os.path.join(propilot_path, "dim_top_levels_202101281645.csv"), sep=";")
df_tree_nodes = pd.read_csv(os.path.join(propilot_path, "dim_tree_nodes_202101281645.csv"), sep=";")
df_facts_financials = pd.read_csv(os.path.join(propilot_path, "fact_financials_202101281645.csv"), sep=";")
df_property_values = pd.read_csv(os.path.join(propilot_path, "fact_property_values_202101281645.csv"), sep=";")

In [31]:
df = (df_facts_financials
      .merge(df_tree_nodes, left_on="tree_node_id", right_on="tree_node_id")
      .merge(df_effects, left_on="effect_id", right_on="effect_id")
      .merge(df_states, left_on="state_id", right_on="state_id")
      .merge(df_periods, left_on="period_id", right_on="period_id")
      .merge(df_structures, left_on="structure_id", right_on="structure_id"))

In [32]:
cols = ["tree_node_name", "structure_name", "effect_id", "state_display_name", "period_date", "financials_cumulated_amount"]

In [57]:
mask = (
    (df.structure_name=="Département") & 
    (df.effect_id == "Nombre de TPE/PME/ETI bénéficiaires - APR") &
    (df.period_date == "2020-12-31 23:59:59")
)

In [67]:
# filter on columns and rows
df=df[cols]
df_final = df.loc[mask].copy()

In [68]:
df_final.shape

(109, 6)

In [69]:
df_final.head()

,tree_node_name,structure_name,effect_id,state_display_name,period_date,financials_cumulated_amount
1276,Ille-et-Vilaine / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,5
1277,Eure-et-Loir / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,2
1278,Hautes-Alpes / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,0
1279,Saint-Martin / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,0
1280,Cher / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,1


In [70]:
df_final[["Departement"]] = df_final["tree_node_name"].apply(lambda x: x.split("/")[0].strip())

In [71]:
df_final.sort_values("financials_cumulated_amount", ascending=False)

,tree_node_name,structure_name,effect_id,state_display_name,period_date,financials_cumulated_amount,Departement
1308,Paris / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,64,Paris
1320,Hauts-de-Seine / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,22,Hauts-de-Seine
1312,Bouches-du-Rhône / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,18,Bouches-du-Rhône
1367,Rhône / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,16,Rhône
1281,Nord / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,15,Nord
...,...,...,...,...,...,...,...
1309,Corse-du-Sud / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,0,Corse-du-Sud
1307,Nouvelle-Calédonie / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,0,Nouvelle-Calédonie
1305,Martinique / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,0,Martinique
1359,Territoire de Belfort / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,0,Territoire de Belfort


In [72]:
df.effect_id.unique()

array(['Nombre de TPE/PME/ETI bénéficiaires - APR',
       'Nombre de TPE/PME/ETI bénéficiaires - SBF',
       'Nombre d’entreprises bénéficiares - SBF',
       "Nombre d'accompagnements dispensés - FUM",
       'Nombre de TPE/PME/ETI bénéficiaires - SAC',
       'Nombre d’entreprises bénéficiares - SAC',
       'Montant total des projets - SAC',
       'Nombre de TPE/PME/ETI bénéficiaires - PIT',
       'Nombre d’entreprises bénéficiares - PIT',
       'Montant total des projets - PIT',
       'Emissions de gaz à effet de serre évitées sur la durée de vie des équipements - PEE',
       'Nombre d’entreprises ayant reçu l’aide - PEE',
       'Montant cumulé de l’investissement total ainsi déclenché - PEE',
       'Nombre de projets soutenus - DGA',
       'Nombre d’entreprises ayant reçu l’aide - EEI',
       'Nombre de PME/TPE - EEI',
       'Nombre de nouveaux projets (nouvelle ligne, extension de ligne et pôle) - TCI',
       'Nombre de projets soutenus - ETH',
       'Montant total 

In [79]:
df_final.loc[df_final.Departement == "Ain"]

,tree_node_name,structure_name,effect_id,state_display_name,period_date,financials_cumulated_amount,Departement
1350,Ain / Assurance prospection,Département,Nombre de TPE/PME/ETI bénéficiaires - APR,Valeur Actuelle,2020-12-31 23:59:59,9,Ain


In [75]:
df.period_date.unique()

array(['2020-12-31 00:00:00', '2020-11-30 00:00:00',
       '2020-12-31 23:59:59', '9999-12-31 23:59:59'], dtype=object)

In [76]:
df.structure_name.unique()

array(['Département', 'Région', 'Mesure'], dtype=object)